In [1]:
from collections import OrderedDict

import torch
import torch.nn as nn
import torch.nn.functional as F

import pytorch3d
import pytorch3d.io
from pytorch3d.structures import Meshes
from pytorch3d.renderer import TexturesVertex

import trimesh

from src.util import make_faces

In [2]:
edges = torch.tensor([
    [ 0, 22,  2],
    [ 2, 22, 23],
    
    [ 3,  7,  2],
    [ 7,  2,  6],
    
    [ 1, 18, 19],
    [ 1, 19,  3],
    
    [ 1, 14, 15],
    [ 1, 14,  0],
    #
    [13, 15, 16],
    [15, 16, 18],
    
    [12, 14, 20],
    [14, 22, 20],
    
    [23, 21,  6],
    [ 6,  4, 21],
    
    [ 7, 19, 17],
    [ 5,  7, 17],
    #
    [11, 16, 17],
    [11,  9, 16],
    
    [8,  9, 13],
    [13, 8, 12],
    
    [10, 4,  5],
    [11, 5, 10],
    
    [10,  8, 21],
    [20, 21,  8],
])

torch.div(edges, 4, rounding_mode='trunc')

tensor([[0, 5, 0],
        [0, 5, 5],
        [0, 1, 0],
        [1, 0, 1],
        [0, 4, 4],
        [0, 4, 0],
        [0, 3, 3],
        [0, 3, 0],
        [3, 3, 4],
        [3, 4, 4],
        [3, 3, 5],
        [3, 5, 5],
        [5, 5, 1],
        [1, 1, 5],
        [1, 4, 4],
        [1, 1, 4],
        [2, 4, 4],
        [2, 2, 4],
        [2, 2, 3],
        [3, 2, 3],
        [2, 1, 1],
        [2, 1, 2],
        [2, 2, 5],
        [5, 5, 2]])

In [3]:
torch.remainder(edges, 4)

tensor([[0, 2, 2],
        [2, 2, 3],
        [3, 3, 2],
        [3, 2, 2],
        [1, 2, 3],
        [1, 3, 3],
        [1, 2, 3],
        [1, 2, 0],
        [1, 3, 0],
        [3, 0, 2],
        [0, 2, 0],
        [2, 2, 0],
        [3, 1, 2],
        [2, 0, 1],
        [3, 3, 1],
        [1, 3, 1],
        [3, 0, 1],
        [3, 1, 0],
        [0, 1, 1],
        [1, 0, 0],
        [2, 0, 1],
        [3, 1, 2],
        [2, 0, 1],
        [0, 1, 0]])

In [25]:
def pairwise(iterable):
    "s -> (s0, s1), (s2, s3), (s4, s5), ..."
    a = iter(iterable)
    return zip(a, a)

l = [0, 1, 2, 3, 4, 5, 6, 7]
for x, y in pairwise(edges.tolist()):
    m = list(set(x + y))
    m.sort()
    a1, a2, b1, b2 = m
    #print('elems', a1, a2, b1, b2)
    tri_a, tri_b = a1 // 4, b1 // 4
    #print('tris', tri_a, tri_b)
    ra1, ra2 = a1 % 4, b1 % 4
    rb1, rb2 = b1 % 4, b2 % 4
    print('remainders', ra1, ra2, rb1, rb2)

remainders 0 2 2 3
remainders 2 2 2 3
remainders 1 2 2 3
remainders 0 2 2 3
remainders 1 0 0 2
remainders 0 0 0 2
remainders 0 1 1 3
remainders 1 1 1 3
remainders 1 0 0 1
remainders 0 0 0 1
remainders 0 2 2 3
remainders 0 0 0 1


In [26]:
[[0, 1], [2, 3]]

[[0, 1], [2, 3]]

In [27]:
[[0, 2], [1, 3]]

[[0, 2], [1, 3]]

In [29]:
[0, 1] in [[0, 2], [1, 3]]

False

In [50]:
def get_edge(tri, pair, n):
    square = torch.arange(n**2).reshape(n, n)
    if pair == [0, 1]:
        r = square[0, :]
    elif pair == [2, 3]:
        r = square[:, 0]
    elif pair == [0, 2]:
        r = square[:, -1]
    elif pair == [1, 3]:
        r = square[-1, :]
    else:
        raise Exception(f'Unknown pair {pair}')
    return (tri * n ** 2 + r).tolist()

get_edge(1, [1, 3], 4)

[28, 29, 30, 31]

In [36]:
square[0, :], square[:, 0], square[:, -1], square[-1, :]

(tensor([0, 1, 2, 3, 4]),
 tensor([ 0,  5, 10, 15, 20]),
 tensor([ 4,  9, 14, 19, 24]),
 tensor([20, 21, 22, 23, 24]))

In [51]:
l1, l2 = (
    [28, 29, 30, 31],
    [ 8,  9, 10, 11])

[[a, b, c] for a, b, c in zip(l1, l1[1:], l2)]
    

[[28, 29, 8], [29, 30, 9], [30, 31, 10]]

In [53]:
[[a, b, c] for a, b, c in zip(l2, l2[1:], l1[1:])]

[[8, 9, 29], [9, 10, 30], [10, 11, 31]]

In [55]:
def get_triangles(l1, l2):
    t1 = [[a, b, c] for a, b, c 
          in zip(l1, l1[1:], l2)]
    t2 = [[a, b, c] for a, b, c 
          in zip(l2, l2[1:], l1[1:])]
    return t1 + t2

get_triangles(l1, l2)

[[28, 29, 8], [29, 30, 9], [30, 31, 10], [8, 9, 29], [9, 10, 30], [10, 11, 31]]